# LLM: 3. Filtering History

If you want to pass only messages that meet specific criteria to the LLM's
context, you can use the `filter_func` parameter in `LLMResponse`.

This parameter expects an instance of `BaseHistoryFilter` with its
abstract `call` method defined.

In [1]:
# installing dependencies
%pip install -q chatsky[llm]==0.10.0 langchain-openai==0.2.8

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from typing import Optional

from chatsky import (
    TRANSITIONS,
    RESPONSE,
    Pipeline,
    Transition as Tr,
    conditions as cnd,
    destinations as dst,
)
from chatsky.core.message import Message
from chatsky.utils.testing import is_interactive_mode
from chatsky.llm import LLM_API
from chatsky.responses.llm import LLMResponse
from chatsky.llm.filters import MessageFilter
from chatsky.core.context import Context
from langchain_openai import ChatOpenAI

openai_api_key = os.getenv("OPENAI_API_KEY")

In [3]:
model = LLM_API(
    ChatOpenAI(model="gpt-4o-mini", api_key=openai_api_key),
    system_prompt="You are a database assistant and must help your user to "
    "recover the demanded data from your memory. Act as a note keeper.",
)

Imagine having a bot for taking notes that accumulates a large dialogue history.
In this example, we will use a simple filtering function to retrieve only the
important messages from such a bot, making the context window usage
more efficient. Here, we can mark notes with the `#important` tag and then ask
the bot to create a summary of all
important messages using the `/remind` command.

If you want to learn more about filters,
refer to the [API reference](../apiref/chatsky.llm.filters.rst).

In order to create a custom filter, we need to inherit either from
`BaseHistoryFilter` or `MessageFilter`.

`BaseHistoryFilter` is a more complex and more customizable version which allows
filtering requests and responses in different ways.

In order to use it, inherit from it and define your `call` method
that returns and instance of the
[Return](../apiref/chatsky.llm.filters.rst#chatsky.llm.filters.Return) enum.

`MessageFilter` is a simple version that does not allow for distinction
between requests and responses: they are treated the same by the filter.

In order to use it, inherit from it and define your `single_message_filter_call`
method that returns bool.

In [4]:
class FilterImportant(MessageFilter):
    def single_message_filter_call(
        self, ctx: Context, message: Optional[Message], llm_model_name: str
    ) -> bool:
        """
        :param ctx: Contex object.
        :param message: Either request or response that is being evaluated for
            filtering out.
        :param llm_model_name: Name of the model that is calling the filter.

        :return: Whether message should be included in the history.
        """
        if (
            message is not None
            and message.text is not None
            and "#important" in message.text.lower()
        ):
            return True
        return False

Another use case for filters is offered by the built-in
[FromModel](../apiref/chatsky.llm.filters.rst#chatsky.llm.filters.FromModel).

This filter allows filtering out all messages on turns where response
was not generated by the currently used model.

Additionally, the `history` parameter in `LLMResponse` allows you to
specify the number of dialogue _turns_ that the model will use as history.
The default value is `5`.

In [5]:
toy_script = {
    "main_flow": {
        "start_node": {
            RESPONSE: Message(""),
            TRANSITIONS: [Tr(dst="greeting_node", cnd=cnd.ExactMatch("Hi"))],
        },
        "greeting_node": {
            RESPONSE: LLMResponse(llm_model_name="note_model", history=0),
            TRANSITIONS: [
                Tr(dst="main_node", cnd=cnd.ExactMatch("Who are you?"))
            ],
        },
        "main_node": {
            RESPONSE: Message(
                "Hi! I am your note-taking assistant. "
                "Just send me your thoughts, and if you need to "
                "rewind a bit, send `/remind`, and I will provide "
                "a summary of your #important notes."
            ),
            TRANSITIONS: [
                Tr(dst="remind_node", cnd=cnd.ExactMatch("/remind")),
                Tr(dst=dst.Current()),
            ],
        },
        "remind_node": {
            RESPONSE: LLMResponse(
                llm_model_name="note_model",
                prompt="Create a bullet list from all the previous "
                "messages tagged with #important.",
                history=15,
                filter_func=FilterImportant(),
            ),
            TRANSITIONS: [Tr(dst="main_node")],
        },
        "fallback_node": {
            RESPONSE: Message("I did not quite understand you..."),
            TRANSITIONS: [Tr(dst="main_node")],
        },
    }
}

In [6]:
pipeline = Pipeline(
    toy_script,
    start_label=("main_flow", "start_node"),
    fallback_label=("main_flow", "fallback_node"),
    models={"note_model": model},
)

if __name__ == "__main__":
    if is_interactive_mode():
        pipeline.run()